In [20]:
import numpy as np
import ngca_theoretical
import math

# Whitening

The goal of whitening is for the dataset to have zero mean, and identity covariance matrix. Let's first see if the existing whitening in the code gives that. Below we apply the whitening, and test the mean. If it is far away from zero, then there is a problem with the whitening.

In [8]:
X = np.loadtxt(fname = "3PhData/DataTst.txt")
X_w = ngca_theoretical.whiten(X)
np.mean(X_w, axis=0)

array([ 0.95129933,  0.03302555,  0.06998988, -0.11573042, -0.01852112,
       -0.07883736,  0.10519115, -0.00388839, -0.12351241,  0.0446299 ,
        0.06660781, -0.04885695])

Obviously there is a problem with the whitening. Let's try to define a correct whitening. 

In [23]:
def center(X):
    c = np.mean(X, axis = 0)
    Xw = X - c
    return Xw, c

def whiten(X):
    X_c, c = center(X)
    U, s, VT = np.linalg.svd(X_c, full_matrices=False)
    W = VT.T * (math.sqrt(X.shape[0]) / s) 
    X_w = U * math.sqrt(X.shape[0]) 
    return X_w, c, W

X_w, c, W = whiten(X)

Notice that the whitening procedure also returns the centering shift, and the transformation $W$ that made the covariance $I_n$. 

After the whitenning we should have:
- The whitenned data $X_W$ should have zero mean, and identity covariance.
- X_w  should be equal to applying W to the right after on X after centering (i.e. $X_w = (X - c)W$).


Let's test these.

In [31]:
np.mean(X_w, axis=0), np.linalg.norm(X_w.T @ X_w / X_w.shape[0] - np.eye(12)), np.linalg.norm(X_w - (X - c) @ W)

(array([ 2.39808173e-17,  1.78247694e-15, -7.57616192e-16, -5.43565193e-16,
         5.73596726e-16, -1.17195142e-15, -1.45927714e-15, -4.06785716e-16,
         7.03437308e-16,  5.55688828e-15,  4.83790785e-15, -8.52651283e-17]),
 2.9550030233451926e-15,
 1.981134203648259e-13)

# NGCA algorithm

Now that we whitenned the data, let's apply the NGCA algorithm.

In [32]:
params = {'alpha1': 0.6754445940381727, 'alpha2': 0.29744739800298886, 
          'beta1': 0.3403472323546272, 'beta2': 0.6441926407645018}
X1 = X[:500]
X2 = X[500:]
approx_NG_subspace = ngca_theoretical.run_ngca_algorithm(X1, X2,
                                                         params['alpha1'], params['alpha2'], 
                                                         params['beta1'], params['beta2'])

**The subspace should be a subspace of $\mathbb{R}^{12}$, so how can the basis below be 500-by-4???**

In [34]:
approx_NG_subspace.shape

(500, 4)

**The problem seems to be in computing the Phi and Psi matrices. They should be 12-by-12, not 500-by-500**.

In [36]:
matrix_phi = ngca_theoretical.compute_matrix_phi(X1, X2, params['alpha1'])
matrix_phi.shape

(500, 500)